In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics

plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [26]:
# used for Alexnet
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
# define samplers for obtaining training and validation batches
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
transform = transforms.Compose([
    transforms.Resize(size=(32, 32)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
training_id =10
model_short_name = 'resnet18'
framework = 'pytorch'

In [9]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/pytorch/resnet18/'

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_exp_resnet_2021-11-06_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

44813453

In [10]:
size1 = os.path.getsize(path+model_name+'.pth')
size1

44813453

In [7]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [8]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/errors/'

In [9]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [10]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    traced_model = torch.jit.trace(model, x)
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    torch_out = model(x)
    t_elapsed_1 = time.time() - since_1
    
    # Export the model
    if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
        os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
    if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
        os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
        
    since_1 = time.time()
    
    since_onnx = time.time()
    torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(to_numpy(torch_out)-ort_outs[0])
    rel_err = np.absolute(to_numpy(torch_out)-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err)
    

    
    ## Converting the coremltool
    since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_5 = time.time() - since_1
    
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    #print(name_1)

    size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    abs_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)
    rel_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)/ np.absolute(coreml_array_output)
    
    for j in range (len(abs_err)):
        for k in range(len(abs_err[j])): 
            data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [11]:
def _lets_convert(data, data_writer_error, data_writer_error2, data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    for i, (images, labels) in enumerate(data):
        to_onnx(i, images,data_writer_error, data_writer_error2, data_writer_run, batch_size)
        if i == 5: 
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [12]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])
    
    #for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150,200, 250,300,350, 400, 450, 500]:
    for batch_size in [1, 5,10,50,100,128, 150,200, 250,300,350, 400, 450, 500]:
        print("################ Batch size: ", batch_size)
        #trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
        valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
        
        _lets_convert(valloader, data_writer_error, data_writer_error2, data_writer_run, batch_size)
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [15]:
run_experiment(model_short_name, model_name)

################ Batch size:  1
converting for batch:  0


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)



*********




Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '313', of the source model, has been renamed to 'var_313' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:03<00:00, 55.46 ops/s] 


converting for batch:  1

*********




...

converting for batch:  4

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:02<00:00, 74.49 ops/s] 


converting for batch:  5

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:02<00:00, 81.06 ops/s] 


Conversion complete in 2m 42s


In [27]:
for i, (images, labels) in enumerate(valloader):
    x_test = images
    y_test = labels
    break

In [31]:
traced_model = torch.jit.trace(model, x_test)

In [32]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
coreml_model = coremltools.convert(
    traced_model,
    inputs=[coremltools.TensorType(shape=x_test.shape)]
 )

Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '313', of the source model, has been renamed to 'var_313' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:01<00:00, 130.13 ops/s]


In [33]:
split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
name_1 = split_[0].replace('name: "', '')
name_1 = name_1.replace('"', '')

# VGG

In [14]:
model_short_name = 'vgg'

In [15]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/pytorch/vgg/'

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_exp_vgg_2021-11-04_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

515298923

In [16]:
size1 = os.path.getsize(path+model_name+'.pth')
size1

515298923

In [17]:
def to_onnx(i, x, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    traced_model = torch.jit.trace(model, x)
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    torch_out = model(x)
    t_elapsed_1 = time.time() - since_1
    
    # Export the model
    if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
        os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
    if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
        os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
        
    since_1 = time.time()
    
    '''since_onnx = time.time()
     torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(to_numpy(torch_out)-ort_outs[0])
    rel_err = np.absolute(to_numpy(torch_out)-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err) '''
    

    
    ## Converting the coremltool
    since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_5 = time.time() - since_1
    
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    #print(name_1)

    size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    abs_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)
    rel_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)/ np.absolute(coreml_array_output)
    
    #for j in range (len(abs_err)):
    #    for k in range(len(abs_err[j])): 
    #        data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    #data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [18]:
def _lets_convert(data, data_writer_error2, data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    for i, (images, labels) in enumerate(data):
        to_onnx(i, images,data_writer_error2, data_writer_run, batch_size)
        if i == 5: 
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [19]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    
    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])
    
    #for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150,200, 250,300,350, 400, 450, 500]:
    for batch_size in [128]:
        print("################ Batch size: ", batch_size)
        #trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
        valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
        
        _lets_convert(valloader,data_writer_error2, data_writer_run, batch_size)
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [20]:
run_experiment(model_short_name, model_name)

################ Batch size:  128
converting for batch:  0


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Converting Frontend ==> MIL Ops:   0%|          | 0/103 [00:00<?, ? ops/s]/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/ops/defs/_utils.py:248: RuntimeWarning: invalid value encountered in long_scalars
  out_shape.append(math.floor((input_shape[r] + pad[r] - effective_ks[r]) / strides[r] + 1))
Converting Frontend ==> MIL Ops:  87%|████████▋ | 90/103 [00:00<00:00, 1351.88 ops/s]


ValueError: cannot convert float NaN to integer

In [28]:
x_test.shape

torch.Size([128, 3, 224, 224])

In [25]:
x.shape

torch.Size([1, 3, 224, 224])

In [29]:
x = Variable(torch.randn(1, 3, 224, 224))
torch.onnx.export(model, x_test, 'vgg16_pytorch.onnx', verbose=True, opset_version=11, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

graph(%input.1 : Float(128, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %classifier.0.weight : Float(4096, 25088, strides=[25088, 1], requires_grad=1, device=cpu),
      %classifier.0.bias : Float(4096, strides=[1], requires_grad=1, device=cpu),
      %classifier.3.weight : Float(4096, 4096, strides=[4096, 1], requires_grad=1, device=cpu),
      %classifier.3.bias : Float(4096, strides=[1], requires_grad=1, device=cpu),
      %classifier.6.weight : Float(10, 4096, strides=[4096, 1], requires_grad=1, device=cpu),
      %classifier.6.bias : Float(10, strides=[1], requires_grad=1, device=cpu),
      %100 : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %101 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %103 : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %104 : Float(128, strides=[1], requires_grad=0, device=cpu),
      %106 : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1]

In [44]:
def torch_pool(inputs, target_size):
    start_points = (torch.arange(target_size, dtype=torch.float32) * (inputs.size(-1) / target_size)).long()
    end_points = ((torch.arange(target_size, dtype=torch.float32)+1) * (inputs.size(-1) / target_size)).ceil().long()
    pooled = []
    for idx in range(target_size):
        pooled.append(torch.mean(inputs[:, :, start_points[idx]:end_points[idx]], dim=-1, keepdim=False))
    pooled = torch.cat(pooled, -1)
    pooled = pooled.unsqueeze(0) 
    return pooled


inps = np.array([0, 1, 2, 3, 4, 5, 6], dtype=np.float32)[None, :, None]
inps_torch = np.transpose(inps, (0, 2, 1))
x = torch_pool(torch.tensor(x_test), 4)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [45]:
x_test

tensor([[[[ 0.2392,  0.2471,  0.2941,  ...,  0.0745, -0.0118, -0.0902],
          [ 0.1922,  0.1843,  0.2471,  ...,  0.0667, -0.0196, -0.0667],
          [ 0.1843,  0.1843,  0.2392,  ...,  0.0902,  0.0196, -0.0588],
          ...,
          [-0.4667, -0.6706, -0.7569,  ..., -0.7020, -0.8980, -0.6863],
          [-0.5216, -0.6157, -0.7255,  ..., -0.7961, -0.7725, -0.8431],
          [-0.5765, -0.5608, -0.6471,  ..., -0.8118, -0.7333, -0.8353]],

         [[-0.1216, -0.1294, -0.0902,  ..., -0.2549, -0.2863, -0.3333],
          [-0.1216, -0.1373, -0.1059,  ..., -0.2549, -0.2863, -0.3098],
          [-0.1373, -0.1451, -0.1294,  ..., -0.2314, -0.2549, -0.3020],
          ...,
          [-0.0275, -0.2157, -0.3098,  ..., -0.2392, -0.4980, -0.3333],
          [-0.0902, -0.2000, -0.3333,  ..., -0.3569, -0.3569, -0.4980],
          [-0.1608, -0.1765, -0.3020,  ..., -0.3961, -0.3412, -0.4745]],

         [[-0.6157, -0.6314, -0.6000,  ..., -0.7176, -0.7176, -0.7412],
          [-0.6000, -0.6863, -

In [46]:
x

tensor([[[[ 0.2098,  0.1983,  0.1444,  ..., -0.6252, -0.6179, -0.6404],
          [-0.1485, -0.1505, -0.1828,  ..., -0.2529, -0.2549, -0.2885],
          [-0.6610, -0.6586, -0.6547,  ...,  0.0696,  0.0566,  0.0096]],

         [[ 0.8221,  0.8468,  0.8350,  ..., -0.4262, -0.3434, -0.2150],
          [ 0.8223,  0.8466,  0.8360,  ..., -0.4255, -0.3169, -0.1740],
          [ 0.8233,  0.8461,  0.8368,  ..., -0.4836, -0.4088, -0.3061]],

         [[ 0.6515,  0.7032,  0.7132,  ..., -0.7505, -0.7635, -0.7939],
          [ 0.7483,  0.7833,  0.7730,  ..., -0.6502, -0.6676, -0.7167],
          [ 0.8529,  0.8674,  0.8321,  ..., -0.5985, -0.6150, -0.6650]],

         ...,

         [[ 0.3934,  0.4203,  0.4118,  ..., -0.5547, -0.5679, -0.6069],
          [ 0.4578,  0.4789,  0.4676,  ..., -0.5630, -0.5961, -0.6010],
          [ 0.4064,  0.4255,  0.4120,  ..., -0.6917, -0.7292, -0.7348]],

         [[ 0.0931, -0.0721, -0.1287,  ..., -0.1404, -0.1551, -0.2324],
          [ 0.1821,  0.0453, -0.0088,  ..

In [47]:
torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 250, 3, 32] to have 3 channels, but got 250 channels instead

In [15]:
for i, (images, labels) in enumerate(valloader):
    x_test = images
    y_test = labels
    break

In [26]:
traced_model = torch.jit.trace(model, x_test)

In [33]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
#coreml_model = coremltools.convert(
#    traced_model,
#    inputs=[coremltools.TensorType(shape=x_test.shape)]
#)

coreml_model = coremltools.convert(traced_model, inputs=[coremltools.TensorType(shape=x_test.shape)],
                       image_input_names=['0'], 
                       image_output_names=['186'],
                       preprocessing_args={'image_scale':1.0/255.0},
                       deprocessing_args={'image_scale':255.0})

Converting Frontend ==> MIL Ops:  87%|████████▋ | 90/103 [00:00<00:00, 2727.04 ops/s]


ValueError: cannot convert float NaN to integer